In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import json
import re
import pickle

In [3]:
import collections
import math
import os
import random
import zipfile

# ko-Wikipedia

In [4]:
import xml.etree.ElementTree as ET

In [5]:
from xml.dom import minidom

In [6]:
root = ET.parse('../data/kowiki.xml').getroot()

In [28]:
kowiki = pd.DataFrame()

In [47]:
flag = 10

In [49]:
for child in root.findall('{http://www.mediawiki.org/xml/export-0.10/}page'):
    if flag:
        flag -= 1
        continue
    rev = child.find('{http://www.mediawiki.org/xml/export-0.10/}revision')
    for r in rev:
        print (r.tag, r.text)
    break

{http://www.mediawiki.org/xml/export-0.10/}id 15252069
{http://www.mediawiki.org/xml/export-0.10/}parentid 15132558
{http://www.mediawiki.org/xml/export-0.10/}timestamp 2015-11-23T06:00:49Z
{http://www.mediawiki.org/xml/export-0.10/}contributor 
        
{http://www.mediawiki.org/xml/export-0.10/}minor None
{http://www.mediawiki.org/xml/export-0.10/}comment "포털:응용과학"을 "포털:기술"로 고침
{http://www.mediawiki.org/xml/export-0.10/}model wikitext
{http://www.mediawiki.org/xml/export-0.10/}format text/x-wiki
{http://www.mediawiki.org/xml/export-0.10/}text <!-- 머릿글 -->
<div id="mp_header" class="mp_outerbox" style="border:1px solid #a7d7f9; margin: 0 0 0.5em; overflow: hidden; padding: 0.5em; font-size: small; -moz-border-radius: 0.8em; -webkit-border-radius: 0.8em; border-radius:0.8em;">
{| width="100%" 
| <div style="float: left; width: 0em; height: 0em; margin: -0.5em 0em 0em -2.5em; overflow: visiable;">[[파일:Wikipedia-logo-v2-200px-transparent.png|140px|링크=]]</div>
<div style="float: left; mar

# Namu Wiki

In [ ]:
with open('../data/namuwiki.json') as json_file:    
    data = json.load(json_file)

In [ ]:
frame = pd.io.json.json_normalize(data)

## save pickle

In [ ]:
f = open('../data/namuwiki.pick', 'wb')
pickle.dump(frame, f)
f.close()

## load pickle

In [51]:
f = open('../data/namuwiki.pick', 'rb')
frame = pickle.load(f)
f.close()

In [53]:
print(frame.shape)
frame.head(3)

(842910, 4)


,contributors,namespace,text,title
0,"[namubot, R:hoon12560]",0,#redirect 느낌표\n,!
1,"[anatra95, chkong1998, Iviyuki, kirby10, max02...",0,[[파일:3444050440.jpg]]\n([[신 세계수의 미궁 2]]에서 뜬 !!...,!!아앗!!
2,"[ABC, AhnJ2000, aottkd3014, dream33, Fairy, ga...",0,"[include(틀:다른 뜻1, other1=말줄임표 등으로 사용하는 용어, rd1...",“……”


In [ ]:
pat_redirect = re.compile('^#redirect (.+)')
def check_redirect(text):
    match = pat_redirect.match(text)
    if match:
        return match.group(1)
    else:
        return False

In [ ]:
def get_article(title):
    p = frame.loc[frame['title'] == title]
    return p.text.values[0]    

In [ ]:
pat_top = re.compile(r'\* 상위 문서 \: (.+)')
pat_bracket = re.compile(r'\[\[(.+?)\]\]')

def get_top(text):
    matches = pat_top.finditer(text)
    ret = []
    for match in matches:
        bracket = pat_bracket.search(match.group(1))
        if bracket:
            ret.append(bracket.group(1))
        else:
            ret.append(match.group(1))
    return ret

In [ ]:
cate = ['배우/한국', '가수/한국']

In [ ]:
actors = []
for idx, context in frame.iterrows():
    if any(actor in get_top(context.text) for actor in cate):
        actors.append(context.title)

In [ ]:
print (len(actors))
for actor in actors[:5]:
    print (actor)

##  save actors

In [ ]:
with open('../data/actors.txt', 'w') as f:
    for actor in actors:
        f.write(actor + r'\n')

In [ ]:
pat_bracket = re.compile(r'\[\[(.+?)\]\]')
pat_file = re.compile(r'\[\[파일:(.+)\]\]')
pat_link = re.compile(r'\[\[(.+?)\|(.+?)\]\]')
pat_comment = re.compile(r'\[\*(.+?)\]')
pat_high = re.compile(r'\{\{\{(.+?)\}\}\}')

def bracket_filter(text):
    ret = ""
    match = pat_file.match(text)
    if match: 
        ret = ""
    else:
        match = pat_link.match(text)
        if match: 
            ret = match.group(2)
        else:
            ret = text[2:-2]
    return ret

def context_filter(text):
    # find bracket
    delc = 0
    matches = pat_bracket.finditer(text)
    for match in matches:
        conv = bracket_filter(match.group(0))
        text = text[:match.start() - delc] + conv +  text[match.end() - delc:]
        delc += len(match.group(0)) - len(conv)
        
    # comments
    delc = 0
    matches = pat_comment.finditer(text)
    for match in matches:
        text = text[:match.start() - delc] + match.group(0) +  text[match.end() - delc:]
        delc += 3
        
    # find highlight
    delc = 0
    matches = pat_high.finditer(text)
    for match in matches:
        text = text[:match.start() - delc] + match.group(1) +  text[match.end() - delc:]
        delc += 6
    
    return text

In [ ]:
def tokenize(doc):
    return ['/'.join(t) for t in Twitter.pos(doc)]

In [ ]:
def rawnize(title):
    return get_article(title)

In [ ]:
def textnize(title):
    text = get_article(title)
    return context_filter(text)

In [ ]:
for idx, db in frame.iterrows():
    text = db.text
    chk = check_redirect(text)
    if chk:
        text = get_article(chk)
    context = context_filter(text)
    print (context)
    break